In [1]:
import sys  ###mclglobal1（下のセルも）
sys.path.append('../scripts/')
from mcl import *

In [2]:
class GlobalMcl(Mcl):  
    def __init__(self, envmap, num, motion_noise_stds={"nn":0.19, "no":0.001, "on":0.13, "oo":0.2},\
                 distance_dev_rate=0.14, direction_dev=0.05): #姿勢の引数を消す
        super().__init__(envmap, np.array([0, 0, 0]).T, num, motion_noise_stds, distance_dev_rate, direction_dev) #初期姿勢は適当
        for p in self.particles: #ランダムに姿勢を初期化し直す，-5～5mの範囲
            p.pose = np.array([np.random.uniform(-5.0, 5.0), np.random.uniform(-5.0, 5.0), np.random.uniform(-math.pi, math.pi)]).T

In [3]:
def trial(animation): ###mclglobal1test
    time_interval = 0.1
    world = World(30, time_interval, debug=not animation)  #アニメーションのON、OFFをdebugで制御

    ## 地図を生成して3つランドマークを追加 ##
    m = Map()
    for ln in [(-4,2), (2,-3), (3,3)]: m.append_landmark(Landmark(*ln))
    world.append(m)

    ## ロボットを作る ##
    init_pose = np.array([np.random.uniform(-5.0, 5.0), np.random.uniform(-5.0, 5.0), np.random.uniform(-math.pi, math.pi)]).T
    pf = GlobalMcl(m, 100)
    a = EstimationAgent(time_interval, 0.2, 10.0/180*math.pi, pf)
    r = Robot(init_pose, sensor=Camera(m), agent=a, color="red")
    world.append(r)

    world.draw()
    
    return (r.pose, pf.pose) #真の姿勢と推定姿勢を返す

In [4]:
if __name__ == '__main__': ###mclglobal1exec
    ok = 0
    for i in range(10):
        actual, estm = trial(False)
        diff = math.sqrt((actual[0]-estm[0])**2 + (actual[1]-estm[1])**2)
        print(i, "真値:", actual, "推定値", estm, "誤差:", diff)
        if diff <= 1.0:
            ok += 1

    print(ok)

0 真値: [-2.76184437 -3.22887654  4.65147833] 推定値 [-3.0419702  -3.72853926  4.92619714] 誤差: 0.572829218562337
1 真値: [ 2.83547726 -0.64841232  7.99148633] 推定値 [-2.83239370e-03  4.64759996e+00  5.97865499e+00] 誤差: 6.008639426140695
2 真値: [-3.94536644  0.54354896  5.843582  ] 推定値 [ 0.78988195 -1.76445802  6.80968785] 誤差: 5.267776912003401
3 真値: [-4.27381482  1.44569979  6.9778344 ] 推定値 [-3.33563556  1.80464215  2.54964628] 誤差: 1.0044998490763708
4 真値: [4.14994389 1.26286677 1.8902833 ] 推定値 [4.55738637 1.75490726 2.21901906] 誤差: 0.6388373910902841
5 真値: [ 4.10635651 -2.02121904  4.30430429] 推定値 [ 4.10173925 -1.06227885  4.59760627] 誤差: 0.9589513031911198
6 真値: [0.94438259 3.67389806 3.25157696] 推定値 [0.74888724 3.84267832 3.28092858] 誤差: 0.2582735220494458
7 真値: [ 4.19678885 -2.21614392  7.84338456] 推定値 [-2.75100087  0.13247263  6.59664426] 誤差: 7.334015388547693
8 真値: [-4.53201679 -0.23177988  4.45129032] 推定値 [-1.28322102  4.28074599  3.81693538] 誤差: 5.560356435258483
9 真値: [-1.28550099 -4.53